In [1]:
import gensim
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import Word2Vec
import torch

In [4]:
df=pd.read_table('labeledTrainData.tsv')
df

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
787,2158_10,1,"I don't like \grade inflation\"" but I just had..."
788,4035_10,1,In a word...amazing.<br /><br />I initially wa...
789,7563_1,0,"Unfortunately, this movie does no credit whats..."
790,6280_7,1,If you like cars you will love this film!<br /...


In [5]:
reviewSeries=df.review.apply(gensim.utils.simple_preprocess)  #preprocessing out review text using gensim
reviewSeries


0      [with, all, this, stuff, going, down, at, the,...
1      [the, classic, war, of, the, worlds, by, timot...
2      [the, film, starts, with, manager, nicholas, b...
3      [it, must, be, assumed, that, those, who, prai...
4      [superbly, trashy, and, wondrously, unpretenti...
                             ...                        
787    [don, like, grade, inflation, but, just, had, ...
788    [in, word, amazing, br, br, initially, was, no...
789    [unfortunately, this, movie, does, no, credit,...
790    [if, you, like, cars, you, will, love, this, f...
791    [warning, may, contain, spoilers, br, br, so, ...
Name: review, Length: 792, dtype: object

In [6]:
model = Word2Vec(sentences=reviewSeries, vector_size=100, window=10, min_count=1, workers=4)
model.save("word2vec.model")

In [7]:
Y=df.sentiment.to_numpy()

In [47]:
x_train,x_test,y_train,y_test=train_test_split(reviewSeries,Y,test_size=.6,random_state=10)

In [62]:
a,x__test,a,y__test=train_test_split(x_test,y_test,test_size=.2,random_state=10)

In [94]:
class GRU():
    def __init__(self):
        self.H=torch.zeros(20,1)
        self.W_z=torch.rand(20,120,dtype=torch.float64,requires_grad=True)
        self.b_z=torch.rand(20,1,requires_grad=True)
        self.W_r=torch.rand(20,120,dtype=torch.float64,requires_grad=True)
        self.b_r=torch.rand(20,1,requires_grad=True)
        self.W_h=torch.rand(20,120,dtype=torch.float64,requires_grad=True)
        self.b_h=torch.rand(20,1,requires_grad=True)
        self.W_fi=torch.rand(1,20,dtype=torch.float64,requires_grad=True)
        self.W_ci=torch.rand(1,1,requires_grad=True)

        self.MSE=torch.nn.MSELoss()
        self.optimiser=torch.optim.Adam([self.W_z,self.b_z,self.W_r,self.b_r,self.W_h,self.b_h,self.W_fi,self.W_ci],lr=0.001)


    def ForwarPass(self,a):
        temp=self.H
        for word in a:
            vec=model.wv[word]
            vec=torch.tensor(vec,dtype=torch.float64)
            x=vec.reshape(100,-1)
            a_=torch.cat((temp,x),0)
            z=torch.sigmoid(torch.matmul(self.W_z,a_)+self.b_z)

            r=torch.sigmoid(torch.matmul(self.W_r,a_)+self.b_r)

            m=torch.mul(r,temp)
            m_=torch.cat((m,x),0)
            h=torch.tanh(torch.matmul(self.W_h,m_)+self.b_h)

            temp=torch.mul((1-z),temp)+torch.mul(z,h)
        y=torch.matmul(self.W_fi,temp)+self.W_ci
        return y

    def Train(self,X,Y,epoch):
        GruLoss=[]
        for j in range(epoch):
            total_ls =0
            for i,ele in enumerate(X):
                ypre=self.ForwarPass(ele)
                ypre=ypre.reshape(-1)
                los=self.MSE(ypre,Y[i])
                self.optimiser.zero_grad()
                total_ls = total_ls + los.detach().item()

                los.backward()

                self.optimiser.step()

            GruLoss.append(total_ls/len(X))

            print(f' loss at epoch {j} is {total_ls/len(X)}')
        return GruLoss

    def Test(self,X,Y):
        total_ls =0
        for i,ele in enumerate(X):
            ypre=self.ForwarPass(ele)
            if(ypre> .5):
              ypre=1
            else:
              ypre=0

            if(ypre == Y[i].item()):
              total_ls += 1

        return total_ls/len(X)

In [95]:
gru=GRU()

In [96]:
y_train=torch.tensor(y_train,dtype=torch.float64)

<ipython-input-96-cefd9b0be394>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train=torch.tensor(y_train,dtype=torch.float64)


In [98]:
gru.Train(x_train,y_train,5)

 loss at epoch 0 is 0.25971977663248663
 loss at epoch 1 is 0.25716459126681446
 loss at epoch 2 is 0.2551702144024837
 loss at epoch 3 is 0.25347165891862017
 loss at epoch 4 is 0.2520829235441264


[0.25971977663248663,
 0.25716459126681446,
 0.2551702144024837,
 0.25347165891862017,
 0.2520829235441264]

In [99]:
y_test=torch.tensor(y__test,dtype=torch.float64)

In [100]:
score = gru.Test(x__test,y_test)
score

0.53125